In [1]:
import tensorflow as tf
import os
import time
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from nilearn import image
from nilearn import plotting
import nibabel as nib
from nilearn.plotting import plot_stat_map, plot_anat, plot_img

Matplotlib created a temporary config/cache directory at /tmp/pbs.3533491.pbsha.ib.sockeye/matplotlib-vw_u0qja because the default path (/home/mkhademi/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
output_dir = '/scratch/st-ipor-1/mkhademi/'
batch_size = 37
subjs = ['CSI1', 'CSI2', 'CSI3', 'CSI4'] 
sub = subjs[0]
#dataset = tf.data.TFRecordDataset(filenames = [output_dir + 'image_data/' + sub + 'bold5000_coco.tfrecords']).batch(batch_size)
train_ds = tf.data.TFRecordDataset(filenames = [output_dir + 'image_data/' + sub + 'bold5000_common.tfrecords']).batch(batch_size)
def tf_parse(eg):
    example = tf.io.parse_example(
        eg, {
            'x': tf.io.FixedLenFeature(shape=(71, 89, 72, 1), dtype=tf.float32),
            'yhat': tf.io.FixedLenFeature(shape=(1000), dtype=tf.float32),
            'y_coco': tf.io.FixedLenFeature(shape=(90), dtype=tf.int64),
            'y_imagenet': tf.io.FixedLenFeature(shape=(1000), dtype=tf.int64),
            'y_super': tf.io.FixedLenFeature(shape=(12), dtype=tf.int64),
            'y_common': tf.io.FixedLenFeature(shape=(90+12+1), dtype=tf.int64)
        })
    return example['x'], example['y_common'] 

2022-03-24 08:04:02.439424: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-03-24 08:04:02.496631: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla V100-SXM2-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.53
pciBusID: 0000:86:00.0
2022-03-24 08:04:02.506881: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2022-03-24 08:04:02.566164: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2022-03-24 08:04:02.623236: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2022-03-24 08:04:02.732664: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10.0
2022-03-24 08:04:02.805419: I tensorflow/stream_executor/platfo

In [3]:
def train_sub(train_ds, j):
    n_epochs = 50
    train = train_ds.unbatch().batch(10)
    train = train.shuffle(2864//batch_size)
    train = train.map(tf_parse)
    train = train.map(lambda x, y: (x, tf.concat([y[:,j:j+1], tf.ones_like(y[:,j:j+1], dtype=tf.int64)-y[:,j:j+1]], axis=1))) 
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(71, 89, 72, 1)),
        tf.keras.layers.Conv3D(1, 3, activation='relu', padding='same', name='conv1'),
        #tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
        tf.keras.layers.Conv3D(2, 3, activation='relu', padding='same', name='conv2'),
        tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
        tf.keras.layers.Conv3D(4, 3, padding='same', name='conv3'),
        tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(rate=0.4),
        # tf.keras.layers.Dense(100, activation='sigmoid'),
        tf.keras.layers.Dense(2),
        #tf.keras.layers.Softmax()
        ])
    model.summary()
    model.compile(
        # loss='mse',
        loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        # loss = tf.keras.losses.BinaryCrossentropy(from_logits=False),
        # optimizer=keras.optimizers.RMSprop(),
        optimizer='adam',
        # metrics=[tf.keras.metrics.MeanSquaredError()],
        metrics=[tf.keras.metrics.CategoricalAccuracy()]
    )
    model.fit(train, epochs=n_epochs)
    
    model_rand = tf.keras.Sequential([
        tf.keras.Input(shape=(71, 89, 72, 1)),
        tf.keras.layers.Conv3D(1, 3, activation='relu', padding='same', name='conv1'),
        #tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
        tf.keras.layers.Conv3D(2, 3, activation='relu', padding='same', name='conv2'),
        tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
        tf.keras.layers.Conv3D(4, 3, padding='same', name='conv3'),
        tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(rate=0.4),
        # tf.keras.layers.Dense(100, activation='sigmoid'),
        tf.keras.layers.Dense(2),
        #tf.keras.layers.Softmax()
        ])
    model_rand.compile(
        # loss='mse',
        loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        # loss = tf.keras.losses.BinaryCrossentropy(from_logits=False),
        # optimizer=keras.optimizers.RMSprop(),
        optimizer='adam',
        # metrics=[tf.keras.metrics.MeanSquaredError()],
        metrics=[tf.keras.metrics.CategoricalAccuracy()]
    )
    train_rand = train.map(lambda x, y: (x, tf.random.shuffle(y)))
    model_rand.fit(train_rand, epochs=n_epochs)
    return model, model_rand

In [4]:
def test_sub(test_ds, j, model, model_rand):
    recalls = []
    aucs = []
    accs = []
    recalls_rand = []
    aucs_rand = []
    accs_rand = []
    test = test_ds.map(tf_parse) 
    test = test.map(lambda x, y: (x, tf.concat([y[:,j:j+1], tf.ones_like(y[:,j:j+1], dtype=tf.int64)-y[:,j:j+1]], axis=1)))
    _, acc = model.evaluate(test)
    _, acc_rand = model_rand.evaluate(test)
    model.add(tf.keras.layers.Softmax())
    model_rand.add(tf.keras.layers.Softmax())
    m_recall = tf.keras.metrics.Recall()
    m_auc = tf.keras.metrics.AUC()
    m_recall_rand = tf.keras.metrics.Recall()
    m_auc_rand = tf.keras.metrics.AUC()
    m_acc_rand = tf.keras.metrics.CategoricalAccuracy()

    for x_batch, y_batch in test:
        yhat = model(x_batch)
        yhat_rand = model_rand(x_batch)
        m_recall.update_state(y_batch[:,0], yhat[:,0])
        m_auc.update_state(y_batch[:,0], yhat[:,0])
        m_recall_rand.update_state(y_batch[:,0], yhat_rand[:,0])
        m_auc_rand.update_state(y_batch[:,0], yhat_rand[:,0])
        m_acc_rand.update_state(y_batch, yhat_rand)
    recalls.append(m_recall.result().numpy())
    aucs.append(m_auc.result().numpy())
    accs.append(acc)
    recalls_rand.append(m_recall_rand.result().numpy())
    aucs_rand.append(m_auc_rand.result().numpy())
    accs_rand.append(m_acc_rand.result().numpy())
    print(recalls)
    print('*********')
    print(aucs)
    print('*********')
    print(accs)
    print(recalls_rand)
    print('##########')
    print(aucs_rand)
    print('##########')
    print(accs_rand)

In [5]:
test1_ds = tf.data.TFRecordDataset(filenames=[output_dir + 'image_data/' + subjs[1] + 'bold5000_common.tfrecords']).batch(batch_size)
test2_ds = tf.data.TFRecordDataset(filenames=[output_dir + 'image_data/' + subjs[2] + 'bold5000_common.tfrecords']).batch(batch_size)
test3_ds = tf.data.TFRecordDataset(filenames=[output_dir + 'image_data/' + subjs[3] + 'bold5000_common.tfrecords']).batch(batch_size)

In [6]:
model, model_rand = train_sub(train_ds, 0)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 71, 89, 72, 1)     28        
_________________________________________________________________
conv2 (Conv3D)               (None, 71, 89, 72, 2)     56        
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 35, 44, 36, 2)     0         
_________________________________________________________________
conv3 (Conv3D)               (None, 35, 44, 36, 4)     220       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 17, 22, 18, 4)     0         
_________________________________________________________________
flatten (Flatten)            (None, 26928)             0         
_________________________________________________________________
dropout (Dropout)            (None, 26928)             0

2022-03-24 08:04:09.496198: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2022-03-24 08:04:11.497141: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7


287/287 [==============================] - 16s 57ms/step - loss: 54.4192 - categorical_accuracy: 0.4923
Epoch 2/50


2022-03-24 08:04:25.234653: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:04:25.234716: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 35ms/step - loss: 11.6159 - categorical_accuracy: 0.5548
Epoch 3/50


...

2022-03-24 08:21:00.090632: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:21:00.090672: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 36ms/step - loss: 0.6911 - categorical_accuracy: 0.5538
Epoch 50/50


2022-03-24 08:21:10.534339: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:21:10.534387: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 36ms/step - loss: 0.6896 - categorical_accuracy: 0.5562


2022-03-24 08:21:20.998733: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:21:20.998769: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


In [7]:
test_sub(test1_ds, 0, model, model_rand)

78/78 [==============================] - 7s 86ms/step - loss: 1.1586 - categorical_accuracy: 0.5499


2022-03-24 08:21:27.886721: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:21:27.886772: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


78/78 [==============================] - 5s 70ms/step - loss: 0.6906 - categorical_accuracy: 0.5468


2022-03-24 08:21:33.390773: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:21:33.390830: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.65150344]
*********
[0.5909538]
*********
[0.54993016]
[0.04703161]
##########
[0.5001862]
##########
[0.54678774]


In [8]:
test_sub(test2_ds, 0, model, model_rand)

78/78 [==============================] - 6s 78ms/step - loss: 0.6746 - categorical_accuracy: 0.6062


2022-03-24 08:21:50.446902: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:21:50.446954: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


78/78 [==============================] - 5s 70ms/step - loss: 0.6909 - categorical_accuracy: 0.5430


2022-03-24 08:21:55.960834: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:21:55.960886: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.613724]
*********
[0.63558465]
*********
[0.6062194]
[0.035466462]
##########
[0.49218592]
##########
[0.5429769]


In [9]:
test_sub(test3_ds, 0, model, model_rand)

46/46 [==============================] - 4s 85ms/step - loss: 0.6883 - categorical_accuracy: 0.5526


2022-03-24 08:22:10.657546: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:22:10.657600: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


46/46 [==============================] - 3s 75ms/step - loss: 0.6917 - categorical_accuracy: 0.5391


2022-03-24 08:22:14.141619: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:22:14.141666: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.75191814]
*********
[0.60098964]
*********
[0.5526161]
[0.0370844]
##########
[0.48672527]
##########
[0.5390946]


In [10]:
model, model_rand = train_sub(train_ds, 90)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 71, 89, 72, 1)     28        
_________________________________________________________________
conv2 (Conv3D)               (None, 71, 89, 72, 2)     56        
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 35, 44, 36, 2)     0         
_________________________________________________________________
conv3 (Conv3D)               (None, 35, 44, 36, 4)     220       
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 17, 22, 18, 4)     0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 26928)             0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 26928)            

2022-03-24 08:22:31.620890: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
2022-03-24 08:22:31.620899: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


287/287 [==============================] - 10s 37ms/step - loss: 10.4673 - categorical_accuracy: 0.6253
Epoch 3/50


2022-03-24 08:22:42.100337: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:22:42.100392: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 36ms/step - loss: 5.8804 - categorical_accuracy: 0.6540
Epoch 4/50


...

2022-03-24 08:39:04.290456: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:39:04.290490: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 35ms/step - loss: 0.5933 - categorical_accuracy: 0.7207
Epoch 50/50


2022-03-24 08:39:14.307339: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
2022-03-24 08:39:14.307345: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


287/287 [==============================] - 10s 35ms/step - loss: 0.5922 - categorical_accuracy: 0.7228


2022-03-24 08:39:24.277992: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
2022-03-24 08:39:24.277998: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


In [11]:
test_sub(test1_ds, 90, model, model_rand)

78/78 [==============================] - 5s 69ms/step - loss: 1.1205 - categorical_accuracy: 0.6480


2022-03-24 08:39:29.698008: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:39:29.698059: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


78/78 [==============================] - 5s 69ms/step - loss: 0.6028 - categorical_accuracy: 0.7102


2022-03-24 08:39:35.101467: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_4]]
2022-03-24 08:39:35.101461: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.3379571]
*********
[0.58423966]
*********
[0.6480447]
[0.017654477]
##########
[0.50951135]
##########
[0.71019554]


In [12]:
test_sub(test2_ds, 90, model, model_rand)

78/78 [==============================] - 5s 69ms/step - loss: 0.5889 - categorical_accuracy: 0.7093


2022-03-24 08:39:51.376868: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:39:51.376916: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


78/78 [==============================] - 5s 69ms/step - loss: 0.6224 - categorical_accuracy: 0.7229


2022-03-24 08:39:56.787107: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:39:56.787159: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.11727617]
*********
[0.6039801]
*********
[0.7092942]
[0.0]
##########
[0.49183893]
##########
[0.722921]


In [13]:
test_sub(test3_ds, 90, model, model_rand)

46/46 [==============================] - 3s 70ms/step - loss: 0.6232 - categorical_accuracy: 0.7066


2022-03-24 08:40:11.020820: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[Shape/_6]]
2022-03-24 08:40:11.020826: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


46/46 [==============================] - 3s 70ms/step - loss: 0.6538 - categorical_accuracy: 0.7190


2022-03-24 08:40:14.298834: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:40:14.298883: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.17161018]
*********
[0.6104143]
*********
[0.70664316]
[0.0]
##########
[0.49546275]
##########
[0.71898884]


In [14]:
model, model_rand = train_sub(train_ds, 91)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 71, 89, 72, 1)     28        
_________________________________________________________________
conv2 (Conv3D)               (None, 71, 89, 72, 2)     56        
_________________________________________________________________
max_pooling3d_8 (MaxPooling3 (None, 35, 44, 36, 2)     0         
_________________________________________________________________
conv3 (Conv3D)               (None, 35, 44, 36, 4)     220       
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 17, 22, 18, 4)     0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 26928)             0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 26928)            

2022-03-24 08:40:31.977388: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:40:31.977442: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 11s 37ms/step - loss: 18.5453 - categorical_accuracy: 0.7374
Epoch 3/50


2022-03-24 08:40:42.500146: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:40:42.500202: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 37ms/step - loss: 30.8280 - categorical_accuracy: 0.7413
Epoch 4/50


...

2022-03-24 08:57:24.911860: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:57:24.911896: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 36ms/step - loss: 0.4552 - categorical_accuracy: 0.8321
Epoch 50/50


2022-03-24 08:57:35.277831: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:57:35.277879: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 36ms/step - loss: 0.4513 - categorical_accuracy: 0.8314


2022-03-24 08:57:45.557803: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
2022-03-24 08:57:45.557807: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


In [15]:
test_sub(test1_ds, 91, model, model_rand)

78/78 [==============================] - 6s 78ms/step - loss: 0.7623 - categorical_accuracy: 0.7828


2022-03-24 08:57:51.804489: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:57:51.804548: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


78/78 [==============================] - 5s 70ms/step - loss: 0.4533 - categorical_accuracy: 0.8317


2022-03-24 08:57:57.256529: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:57:57.256599: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.11618257]
*********
[0.58861285]
*********
[0.78282124]
[0.0]
##########
[0.5154317]
##########
[0.8317039]


In [16]:
test_sub(test2_ds, 91, model, model_rand)

78/78 [==============================] - 6s 74ms/step - loss: 0.5345 - categorical_accuracy: 0.7897


2022-03-24 08:58:13.856776: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:58:13.856833: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


78/78 [==============================] - 5s 70ms/step - loss: 0.5328 - categorical_accuracy: 0.8316


2022-03-24 08:58:19.360589: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:58:19.360636: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.10788382]
*********
[0.5506586]
*********
[0.7896576]
[0.0]
##########
[0.48475716]
##########
[0.8315863]


In [17]:
test_sub(test3_ds, 91, model, model_rand)

46/46 [==============================] - 4s 92ms/step - loss: 0.5775 - categorical_accuracy: 0.8225


2022-03-24 08:58:34.360662: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:58:34.360716: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


46/46 [==============================] - 3s 71ms/step - loss: 0.5988 - categorical_accuracy: 0.8372


2022-03-24 08:58:37.688237: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:58:37.688290: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.057761732]
*********
[0.5888153]
*********
[0.8224574]
[0.0]
##########
[0.5102727]
##########
[0.8371546]


In [18]:
model, model_rand = train_sub(train_ds, 92)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 71, 89, 72, 1)     28        
_________________________________________________________________
conv2 (Conv3D)               (None, 71, 89, 72, 2)     56        
_________________________________________________________________
max_pooling3d_12 (MaxPooling (None, 35, 44, 36, 2)     0         
_________________________________________________________________
conv3 (Conv3D)               (None, 35, 44, 36, 4)     220       
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 17, 22, 18, 4)     0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 26928)             0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 26928)            

2022-03-24 08:58:55.027195: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:58:55.027247: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 36ms/step - loss: 12.2253 - categorical_accuracy: 0.8673
Epoch 3/50


2022-03-24 08:59:05.453082: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 08:59:05.453150: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 36ms/step - loss: 5.5877 - categorical_accuracy: 0.8757
Epoch 4/50


...

2022-03-24 09:15:40.175233: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_4]]
2022-03-24 09:15:40.175263: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 36ms/step - loss: 0.2680 - categorical_accuracy: 0.9235
Epoch 50/50


2022-03-24 09:15:50.614995: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:15:50.615041: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 36ms/step - loss: 0.2669 - categorical_accuracy: 0.9228


2022-03-24 09:16:00.970640: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:16:00.970693: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


In [19]:
test_sub(test1_ds, 92, model, model_rand)

78/78 [==============================] - 6s 80ms/step - loss: 0.5280 - categorical_accuracy: 0.9099


2022-03-24 09:16:07.305410: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:16:07.305460: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


78/78 [==============================] - 5s 70ms/step - loss: 0.3117 - categorical_accuracy: 0.9169


2022-03-24 09:16:12.756530: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:16:12.756582: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.013761468]
*********
[0.5223663]
*********
[0.9099162]
[0.0045871558]
##########
[0.4942947]
##########
[0.91689944]


In [20]:
test_sub(test2_ds, 92, model, model_rand)

78/78 [==============================] - 6s 73ms/step - loss: 0.4886 - categorical_accuracy: 0.8197


2022-03-24 09:16:28.994107: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:16:28.994158: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


78/78 [==============================] - 5s 69ms/step - loss: 0.4222 - categorical_accuracy: 0.9235


2022-03-24 09:16:34.422265: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:16:34.422314: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.17431192]
*********
[0.5569924]
*********
[0.8197065]
[0.0]
##########
[0.48665923]
##########
[0.9234801]


In [21]:
test_sub(test3_ds, 92, model, model_rand)

46/46 [==============================] - 4s 81ms/step - loss: 0.5361 - categorical_accuracy: 0.9077


2022-03-24 09:16:48.752360: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:16:48.752410: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


46/46 [==============================] - 3s 70ms/step - loss: 0.5372 - categorical_accuracy: 0.9295


2022-03-24 09:16:52.059741: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:16:52.059789: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.008333334]
*********
[0.50602984]
*********
[0.9077014]
[0.0]
##########
[0.4894766]
##########
[0.92945325]


In [22]:
model, model_rand = train_sub(train_ds, 93)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 71, 89, 72, 1)     28        
_________________________________________________________________
conv2 (Conv3D)               (None, 71, 89, 72, 2)     56        
_________________________________________________________________
max_pooling3d_16 (MaxPooling (None, 35, 44, 36, 2)     0         
_________________________________________________________________
conv3 (Conv3D)               (None, 35, 44, 36, 4)     220       
_________________________________________________________________
max_pooling3d_17 (MaxPooling (None, 17, 22, 18, 4)     0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 26928)             0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 26928)            

2022-03-24 09:17:09.273149: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:17:09.273179: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 36ms/step - loss: 14.3132 - categorical_accuracy: 0.6145
Epoch 3/50


2022-03-24 09:17:19.743914: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:17:19.743977: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 36ms/step - loss: 9.3547 - categorical_accuracy: 0.6379
Epoch 4/50


...

2022-03-24 09:33:31.527883: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:33:31.527937: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 34ms/step - loss: 0.6112 - categorical_accuracy: 0.6983
Epoch 50/50


2022-03-24 09:33:41.414387: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:33:41.414420: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 34ms/step - loss: 0.6123 - categorical_accuracy: 0.6962


2022-03-24 09:33:51.253204: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:33:51.253242: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


In [23]:
test_sub(test1_ds, 93, model, model_rand)

78/78 [==============================] - 5s 68ms/step - loss: 1.3082 - categorical_accuracy: 0.5555


2022-03-24 09:33:56.607441: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:33:56.607496: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


78/78 [==============================] - 5s 68ms/step - loss: 0.6368 - categorical_accuracy: 0.6938


2022-03-24 09:34:01.923556: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:34:01.923608: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.5191193]
*********
[0.5471279]
*********
[0.5555168]
[0.005793743]
##########
[0.48435283]
##########
[0.6937849]


In [24]:
test_sub(test2_ds, 93, model, model_rand)

78/78 [==============================] - 5s 67ms/step - loss: 0.7712 - categorical_accuracy: 0.4815


2022-03-24 09:34:17.963453: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:34:17.963497: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


78/78 [==============================] - 5s 68ms/step - loss: 0.6578 - categorical_accuracy: 0.6939


2022-03-24 09:34:23.286682: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:34:23.286727: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.68677497]
*********
[0.5605136]
*********
[0.4814815]
[0.018561484]
##########
[0.489598]
##########
[0.6939203]


In [25]:
test_sub(test3_ds, 93, model, model_rand)

46/46 [==============================] - 3s 69ms/step - loss: 0.6832 - categorical_accuracy: 0.5567


2022-03-24 09:34:37.200637: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:34:37.200688: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


46/46 [==============================] - 3s 70ms/step - loss: 0.6725 - categorical_accuracy: 0.6949


2022-03-24 09:34:40.462626: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:34:40.462675: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.52964425]
*********
[0.57383776]
*********
[0.55673134]
[0.013833992]
##########
[0.5038782]
##########
[0.6948854]


In [26]:
model, model_rand = train_sub(train_ds, 95)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 71, 89, 72, 1)     28        
_________________________________________________________________
conv2 (Conv3D)               (None, 71, 89, 72, 2)     56        
_________________________________________________________________
max_pooling3d_20 (MaxPooling (None, 35, 44, 36, 2)     0         
_________________________________________________________________
conv3 (Conv3D)               (None, 35, 44, 36, 4)     220       
_________________________________________________________________
max_pooling3d_21 (MaxPooling (None, 17, 22, 18, 4)     0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 26928)             0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 26928)           

2022-03-24 09:34:57.576382: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:34:57.576419: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 35ms/step - loss: 17.4967 - categorical_accuracy: 0.7668
Epoch 3/50


2022-03-24 09:35:07.518755: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:35:07.518802: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 35ms/step - loss: 7.7436 - categorical_accuracy: 0.7709
Epoch 4/50


...

2022-03-24 09:51:18.793195: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
2022-03-24 09:51:18.793211: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


287/287 [==============================] - 10s 36ms/step - loss: 0.4270 - categorical_accuracy: 0.8488
Epoch 50/50


2022-03-24 09:51:29.056235: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
2022-03-24 09:51:29.056251: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


287/287 [==============================] - 10s 36ms/step - loss: 0.4240 - categorical_accuracy: 0.8495


2022-03-24 09:51:39.419874: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:51:39.419917: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


In [27]:
test_sub(test1_ds, 95, model, model_rand)

78/78 [==============================] - 5s 68ms/step - loss: 1.0135 - categorical_accuracy: 0.6072


2022-03-24 09:51:44.775903: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:51:44.775955: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


78/78 [==============================] - 5s 68ms/step - loss: 0.4266 - categorical_accuracy: 0.8495


2022-03-24 09:51:50.127687: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:51:50.127740: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.45475638]
*********
[0.57217324]
*********
[0.60719275]
[0.0]
##########
[0.47070011]
##########
[0.84951115]


In [28]:
test_sub(test2_ds, 95, model, model_rand)

78/78 [==============================] - 5s 68ms/step - loss: 0.7988 - categorical_accuracy: 0.4511


2022-03-24 09:52:06.248741: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:52:06.248789: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


78/78 [==============================] - 5s 68ms/step - loss: 0.5134 - categorical_accuracy: 0.8498


2022-03-24 09:52:11.584918: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:52:11.584970: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.6604651]
*********
[0.5608868]
*********
[0.45108315]
[0.0]
##########
[0.500983]
##########
[0.8497554]


In [29]:
test_sub(test3_ds, 95, model, model_rand)

46/46 [==============================] - 3s 69ms/step - loss: 0.6166 - categorical_accuracy: 0.7284


2022-03-24 09:52:25.599315: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:52:25.599362: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


46/46 [==============================] - 3s 69ms/step - loss: 0.5902 - categorical_accuracy: 0.8519


2022-03-24 09:52:28.832735: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[Shape/_6]]
2022-03-24 09:52:28.832778: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.19444445]
*********
[0.50937843]
*********
[0.72839504]
[0.0]
##########
[0.49799395]
##########
[0.8518519]


In [30]:
model, model_rand = train_sub(train_ds, 97)

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 71, 89, 72, 1)     28        
_________________________________________________________________
conv2 (Conv3D)               (None, 71, 89, 72, 2)     56        
_________________________________________________________________
max_pooling3d_24 (MaxPooling (None, 35, 44, 36, 2)     0         
_________________________________________________________________
conv3 (Conv3D)               (None, 35, 44, 36, 4)     220       
_________________________________________________________________
max_pooling3d_25 (MaxPooling (None, 17, 22, 18, 4)     0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 26928)             0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 26928)           

2022-03-24 09:52:46.337509: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
2022-03-24 09:52:46.337514: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


287/287 [==============================] - 10s 36ms/step - loss: 4.7008 - categorical_accuracy: 0.8307
Epoch 3/50


2022-03-24 09:52:56.554525: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 09:52:56.554573: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 36ms/step - loss: 1.7871 - categorical_accuracy: 0.8516
Epoch 4/50


...

2022-03-24 10:09:29.715981: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
2022-03-24 10:09:29.715993: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


287/287 [==============================] - 10s 36ms/step - loss: 0.3202 - categorical_accuracy: 0.9005
Epoch 50/50


2022-03-24 10:09:40.095345: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
2022-03-24 10:09:40.095360: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


287/287 [==============================] - 10s 36ms/step - loss: 0.3263 - categorical_accuracy: 0.9005


2022-03-24 10:09:50.393295: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
2022-03-24 10:09:50.393300: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


In [31]:
test_sub(test1_ds, 97, model, model_rand)

78/78 [==============================] - 5s 68ms/step - loss: 1.8991 - categorical_accuracy: 0.8457
      1/Unknown - 0s 154ms/step - loss: 0.2928 - categorical_accuracy: 0.9189

2022-03-24 10:09:55.777517: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:09:55.777562: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


78/78 [==============================] - 5s 66ms/step - loss: 0.3379 - categorical_accuracy: 0.9008


2022-03-24 10:10:00.968674: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:10:00.968725: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.1056338]
*********
[0.5203591]
*********
[0.8456704]
[0.0]
##########
[0.46935323]
##########
[0.900838]


In [32]:
test_sub(test2_ds, 97, model, model_rand)

78/78 [==============================] - 5s 67ms/step - loss: 0.4595 - categorical_accuracy: 0.8532


2022-03-24 10:10:17.000981: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:10:17.001030: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


78/78 [==============================] - 5s 67ms/step - loss: 0.4340 - categorical_accuracy: 0.9008


2022-03-24 10:10:22.239583: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:10:22.239636: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.08098592]
*********
[0.52994186]
*********
[0.8532495]
[0.0]
##########
[0.50285053]
##########
[0.9007687]


In [33]:
test_sub(test3_ds, 97, model, model_rand)

46/46 [==============================] - 3s 68ms/step - loss: 0.5535 - categorical_accuracy: 0.8648


2022-03-24 10:10:36.188865: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:10:36.188906: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


46/46 [==============================] - 3s 67ms/step - loss: 0.5512 - categorical_accuracy: 0.8907


2022-03-24 10:10:39.321427: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:10:39.321476: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.032432433]
*********
[0.52428323]
*********
[0.86478543]
[0.0]
##########
[0.50001246]
##########
[0.89065254]


In [34]:
model, model_rand = train_sub(train_ds, 98)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 71, 89, 72, 1)     28        
_________________________________________________________________
conv2 (Conv3D)               (None, 71, 89, 72, 2)     56        
_________________________________________________________________
max_pooling3d_28 (MaxPooling (None, 35, 44, 36, 2)     0         
_________________________________________________________________
conv3 (Conv3D)               (None, 35, 44, 36, 4)     220       
_________________________________________________________________
max_pooling3d_29 (MaxPooling (None, 17, 22, 18, 4)     0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 26928)             0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 26928)           

2022-03-24 10:10:56.780999: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:10:56.781047: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 36ms/step - loss: 11.1674 - categorical_accuracy: 0.7737
Epoch 3/50


2022-03-24 10:11:07.185783: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:11:07.185839: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 36ms/step - loss: 6.5207 - categorical_accuracy: 0.7814
Epoch 4/50


...

2022-03-24 10:27:40.251483: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:27:40.251525: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 36ms/step - loss: 0.4284 - categorical_accuracy: 0.8485
Epoch 50/50


2022-03-24 10:27:50.580395: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:27:50.580440: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 36ms/step - loss: 0.4245 - categorical_accuracy: 0.8478


2022-03-24 10:28:00.872707: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:28:00.872766: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


In [35]:
test_sub(test1_ds, 98, model, model_rand)

78/78 [==============================] - 5s 68ms/step - loss: 1.0668 - categorical_accuracy: 0.6351
      1/Unknown - 0s 177ms/step - loss: 0.5504 - categorical_accuracy: 0.7838

2022-03-24 10:28:06.293390: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:28:06.293434: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


78/78 [==============================] - 5s 68ms/step - loss: 0.4328 - categorical_accuracy: 0.8485


2022-03-24 10:28:11.633617: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:28:11.633664: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.3770115]
*********
[0.54462886]
*********
[0.6351257]
[0.0022988506]
##########
[0.50325286]
##########
[0.8484637]


In [36]:
test_sub(test2_ds, 98, model, model_rand)

78/78 [==============================] - 5s 68ms/step - loss: 0.5212 - categorical_accuracy: 0.7991


2022-03-24 10:28:27.618043: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:28:27.618089: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


78/78 [==============================] - 5s 68ms/step - loss: 0.5029 - categorical_accuracy: 0.8480


2022-03-24 10:28:32.937304: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:28:32.937359: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.085057475]
*********
[0.54766303]
*********
[0.7990915]
[0.0]
##########
[0.5111793]
##########
[0.8480084]


In [37]:
test_sub(test3_ds, 98, model, model_rand)

46/46 [==============================] - 3s 68ms/step - loss: 0.6627 - categorical_accuracy: 0.6149


2022-03-24 10:28:47.051478: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:28:47.051525: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


46/46 [==============================] - 3s 69ms/step - loss: 0.5823 - categorical_accuracy: 0.8424


2022-03-24 10:28:50.293565: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:28:50.293608: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.37686568]
*********
[0.54682535]
*********
[0.6149324]
[0.0]
##########
[0.45618597]
##########
[0.8424456]


In [38]:
model, model_rand = train_sub(train_ds, 102)

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 71, 89, 72, 1)     28        
_________________________________________________________________
conv2 (Conv3D)               (None, 71, 89, 72, 2)     56        
_________________________________________________________________
max_pooling3d_32 (MaxPooling (None, 35, 44, 36, 2)     0         
_________________________________________________________________
conv3 (Conv3D)               (None, 35, 44, 36, 4)     220       
_________________________________________________________________
max_pooling3d_33 (MaxPooling (None, 17, 22, 18, 4)     0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 26928)             0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 26928)           

2022-03-24 10:29:07.461520: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_4]]
2022-03-24 10:29:07.461556: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 36ms/step - loss: 5.5748 - categorical_accuracy: 0.7367
Epoch 3/50


2022-03-24 10:29:17.911765: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:29:17.911805: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 36ms/step - loss: 5.2003 - categorical_accuracy: 0.7598
Epoch 4/50


...

2022-03-24 10:45:50.620009: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:45:50.620070: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 36ms/step - loss: 0.4649 - categorical_accuracy: 0.8247
Epoch 50/50


2022-03-24 10:46:00.927153: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:46:00.927151: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


287/287 [==============================] - 10s 36ms/step - loss: 0.4637 - categorical_accuracy: 0.8251


2022-03-24 10:46:11.259570: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
2022-03-24 10:46:11.259588: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


In [39]:
test_sub(test1_ds, 102, model, model_rand)

78/78 [==============================] - 5s 68ms/step - loss: 1.2962 - categorical_accuracy: 0.6966


2022-03-24 10:46:16.694940: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:46:16.694993: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


78/78 [==============================] - 5s 68ms/step - loss: 0.4641 - categorical_accuracy: 0.8251


2022-03-24 10:46:22.016177: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:46:22.016232: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.22954091]
*********
[0.50488436]
*********
[0.6965782]
[0.0]
##########
[0.49943024]
##########
[0.82506984]


In [40]:
test_sub(test2_ds, 102, model, model_rand)

78/78 [==============================] - 5s 68ms/step - loss: 0.5920 - categorical_accuracy: 0.7061


2022-03-24 10:46:38.143963: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
2022-03-24 10:46:38.143963: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[Shape/_6]]


78/78 [==============================] - 6s 71ms/step - loss: 0.5344 - categorical_accuracy: 0.8249


2022-03-24 10:46:43.696490: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:46:43.696551: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.21556886]
*********
[0.52407676]
*********
[0.7061495]
[0.0]
##########
[0.49879953]
##########
[0.8249476]


In [41]:
test_sub(test3_ds, 102, model, model_rand)

46/46 [==============================] - 3s 69ms/step - loss: 0.6357 - categorical_accuracy: 0.6778


2022-03-24 10:46:57.761329: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:46:57.761380: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


46/46 [==============================] - 3s 69ms/step - loss: 0.6056 - categorical_accuracy: 0.8195


2022-03-24 10:47:00.979233: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-03-24 10:47:00.979283: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.22149837]
*********
[0.51753443]
*********
[0.67783654]
[0.0]
##########
[0.48739365]
##########
[0.8195179]
